In [16]:
import collections
import http.client
import pandas as pd
from operator import itemgetter
import json
from collections import namedtuple
pd.set_option("display.max_rows", 200)
pd.set_option('display.max_colwidth', -1)

In [17]:
Rounds = namedtuple('Rounds', 'player round score strokes thru')

In [18]:
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [19]:
missed_cut = ["Sandy Lyle", "Charl Schwartzel", "Jose Maria Olazabal", "Jason Dufner", "Patrick Cantlay", 
              "Shubhankar Sharma", "Thomas Pieters", "Ted Potter", "Brendan Steele", "Dylan Frittelli", 
              "Danny Willett", "Ross Fisher", "Wesley Bryan", "Pat Perez", "Patton Kizzire", "Trevor Immelman", 
              "Joaquin Niemann", "Yuta Ikeda", "Alexander Noren", "Kevin Chappell", "Doc Redman", "Gary Woodland", 
              "Yusaku Miyazato", "Austin Cook", "Billy Horschel", "Ian Woosnam", "Mike Weir", "Larry Mize", 
              "Angel Cabrera", "Yuxin Lin", "Mark O'Meara", "Matt Parziale", "Sergio Garcia", "Harry Ellis"]
missed_cut_players = [p.lower().replace(" ", "") for p in missed_cut]
# sorted(missed_cut_players)

In [20]:
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df[["first_name", "last_name", "position", "score", "strokes", "rounds", "full_name"]]

,first_name,last_name,position,score,strokes,rounds,full_name
0,Patrick,Reed,1,-11,180,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eagles': 0, 'birdies': 5, 'pars': 11, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': -6, 'strokes': 66, 'thru': 18, 'eagles': 0, 'birdies': 9, 'pars': 6, 'bogeys': 3, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': -2, 'strokes': 45, 'thru': 12, 'eagles': 0, 'birdies': 4, 'pars': 6, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",patrickreed
1,Rory,McIlroy,2,-9,187,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eagles': 0, 'birdies': 4, 'pars': 13, 'bogeys': 1, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': -1, 'strokes': 71, 'thru': 18, 'eagles': 0, 'birdies': 4, 'pars': 11, 'bogeys': 3, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': -5, 'strokes': 47, 'thru': 13, 'eagles': 1, 'birdies': 3, 'pars': 9, 'bogeys': 0, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",rorymcilroy
2,Jon,Rahm,3,-8,208,"[{'score': 3, 'strokes': 75, 'thru': 18, 'eagles': 0, 'birdies': 3, 'pars': 10, 'bogeys': 4, 'double_bogeys': 1, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': -4, 'strokes': 68, 'thru': 18, 'eagles': 1, 'birdies': 4, 'pars': 11, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': -7, 'strokes': 65, 'thru': 18, 'eagles': 1, 'birdies': 5, 'pars': 12, 'bogeys': 0, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",jonrahm
3,Rickie,Fowler,3,-8,197,"[{'score': -2, 'strokes': 70, 'thru': 18, 'eagles': 0, 'birdies': 4, 'pars': 12, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': 0, 'strokes': 72, 'thru': 18, 'eagles': 0, 'birdies': 3, 'pars': 12, 'bogeys': 3, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': -6, 'strokes': 55, 'thru': 15, 'eagles': 1, 'birdies': 4, 'pars': 10, 'bogeys': 0, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",rickiefowler
4,Tommy,Fleetwood,5,-6,210,"[{'score': 0, 'strokes': 72, 'thru': 18, 'eagles': 0, 'birdies': 5, 'pars': 8, 'bogeys': 5, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': 0, 'strokes': 72, 'thru': 18, 'eagles': 0, 'birdies': 3, 'pars': 12, 'bogeys': 3, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': -6, 'strokes': 66, 'thru': 18, 'eagles': 0, 'birdies': 7, 'pars': 10, 'bogeys': 1, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",tommyfleetwood
5,Marc,Leishman,5,-6,185,"[{'score': -2, 'strokes': 70, 'thru': 18, 'eagles': 0, 'birdies': 4, 'pars': 13, 'bogeys': 0, 'double_bogeys': 1, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': -5, 'strokes': 67, 'thru': 18, 'eagles': 1, 'birdies': 4, 'pars': 12, 'bogeys': 1, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': 1, 'strokes': 48, 'thru': 12, 'eagles': 0, 'birdies': 0, 'pars': 11, 'bogeys': 1, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",marcleishman
6,Bubba,Watson,5,-6,202,"[{'score': 1, 'strokes': 73, 'thru': 18, 'eagles': 0, 'birdies': 1, 'pars': 15, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 1}, {'score': -3, 'strokes': 69, 'thru': 18, 'eagles': 0, 'birdies': 5, 'pars': 11, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 2}, {'score': -4, 'strokes': 60, 'thru': 16, 'eagles': 0, 'birdies': 7, 'pars': 6, 'bogeys': 3, 'double_bogeys': 0, 'other_scores': 0, 'holes_in_one': 0, 'sequence': 3}]",bubbawatson
7,Henrik,Stenson,5,-6,190,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eagles': 0, 'birdies': 5, 'pars': 11, 'bogeys': 2, 'double_bogeys': 0, 'other_scores': 0, 'holes

In [21]:
def get_high_round(rounds):
    strokes = max(rounds, key=lambda x: x.strokes)
    return strokes
    
def get_score(player):
    player = player.lower().replace(" ", "")
    score = lb_df.loc[lb_df["full_name"] == player]["score"]
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    high_round = get_high_round(active_rounds)
    if player in missed_cut_players:
        return "**MC** Score: {} Strokes: {}".format(int(score) + high_round.score, int(strokes) + high_round.strokes)
    return "Score: {}, Strokes: {}".format(int(score), int(strokes))
    
def get_strokes(player):
    player = player.lower().replace(" ", "")
    high_strokes = 0
    if player in missed_cut_players:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"]) + get_high_round(active_rounds).strokes
    else:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"])

In [22]:
players_and_rounds = [("{} {}".format(r["first_name"], r["last_name"]), 
                       r["rounds"][2]) for r in json.loads(d)["leaderboard"] if len(r["rounds"]) > 2]
active_rounds = [Rounds(p[0], p[1]["sequence"], p[1]["score"], p[1]["strokes"], p[1]["thru"]) for p in players_and_rounds]

In [23]:
sorted(active_rounds, key=lambda x: x.strokes, reverse=True)

[Rounds(player='Vijay Singh', round=3, score=7, strokes=79, thru=18),
 Rounds(player='Brian Harman', round=3, score=4, strokes=76, thru=18),
 Rounds(player='Kyle Stanley', round=3, score=3, strokes=75, thru=18),
 Rounds(player='Chez Reavie', round=3, score=3, strokes=75, thru=18),
 Rounds(player='Zach Johnson', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Rafael Cabrera-Bello', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Branden Grace', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Martin Kaymer', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Doug Ghim', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Phil Mickelson', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Ian Poulter', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Fred Couples', round=3, score=1, strokes=73, thru=18),
 Rounds(player='Tyrrell Hatton', round=3, score=1, strokes=73, thru=18),
 Rounds(player='Bernd Wiesberger', round=3, score=0, strokes=72, thru

In [24]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player1", "Player2", "Player3", "Player4", 
         "Player5", "Player6", "Player7", "Total", "Tie Breaker"]]

In [25]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:8]:
        score_accum += get_strokes(player)
    df.set_value(index, "Total", score_accum)

In [26]:
df = df.sort_values(by=['Total']).reset_index(drop=True)
for i in range(1,8):
    df["Score{}".format(i)] = df["Player{}".format(i)].apply(lambda x: get_score(x))

In [27]:
high_round = get_high_round(active_rounds)
print("High round of the day belongs to {}, with score: {}, strokes: {}, thru: {}".format(
    high_round.player, high_round.score, high_round.strokes, high_round.thru))

High round of the day belongs to Vijay Singh, with score: 7, strokes: 79, thru: 18


In [28]:
df.index += 1
df = df[["Pool Player", 
         "Player1", "Score1",
         "Player2", "Score2",
         "Player3", "Score3",
         "Player4", "Score4",
         "Player5", "Score5",
         "Player6", "Score6",
         "Player7", "Score7",
         "Total", "Tie Breaker"]]
df

,Pool Player,Player1,Score1,Player2,Score2,Player3,Score3,Player4,Score4,Player5,Score5,Player6,Score6,Player7,Score7,Total,Tie Breaker
1,Dave Peterson,Rory McIlroy,"Score: -9, Strokes: 187",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -11, Strokes: 180",Tony Finau,"Score: -1, Strokes: 199",Jason Dufner,**MC** Score: 13 Strokes: 229,Tiger Woods,"Score: 4, Strokes: 220",Martin Kaymer,"Score: 5, Strokes: 221",1451.0,278.0
2,Joe Kearney,Justin Thomas,"Score: -4, Strokes: 196",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -11, Strokes: 180",Louis Oosthuizen,"Score: -3, Strokes: 202",Thomas Pieters,**MC** Score: 14 Strokes: 230,Tiger Woods,"Score: 4, Strokes: 220",Russell Henley,"Score: 0, Strokes: 216",1459.0,279.0
3,Tristen Woods,Jon Rahm,"Score: -8, Strokes: 208",Henrik Stenson,"Score: -6, Strokes: 190",Patrick Reed,"Score: -11, Strokes: 180",Brendan Steele,**MC** Score: 14 Strokes: 230,Cameron Smith,"Score: -4, Strokes: 208",Billy Horschel,**MC** Score: 18 Strokes: 234,Adam Scott,"Score: 2, Strokes: 218",1468.0,279.0
4,Zachary Sight,Justin Thomas,"Score: -4, Strokes: 196",Phil Mickelson,"Score: 7, Strokes: 223",Patrick Reed,"Score: -11, Strokes: 180",Daniel Berger,"Score: 2, Strokes: 218",Siwoo Kim,"Score: 0, Strokes: 216",Tiger Woods,"Score: 4, Strokes: 220",Fred Couples,"Score: 3, Strokes: 219",1472.0,278.0
5,Jeremy Szyba #3,Dustin Johnson,"Score: -4, Strokes: 196",Phil Mickelson,"Score: 7, Strokes: 223",Patrick Reed,"Score: -11, Strokes: 180",Branden Grace,"Score: 4, Strokes: 220",Adam Hadwin,"Score: 0, Strokes: 216",Daniel Berger,"Score: 2, Strokes: 218",Ian Poulter,"Score: 7, Strokes: 223",1476.0,278.0
6,Kyle Ritchie,Rory McIlroy,"Score: -9, Strokes: 187",Phil Mickelson,"Score: 7, Strokes: 223",Bubba Watson,"Score: -6, Strokes: 202",Louis Oosthuizen,"Score: -3, Strokes: 202",Jason Dufner,**MC** Score: 13 Strokes: 229,Tiger Woods,"Score: 4, Strokes: 220",Russell Henley,"Score: 0, Strokes: 216",1479.0,277.0
7,Jason Grill,Dustin Johnson,"Score: -4, Strokes: 196",Jason Day,"Score: -1, Strokes: 215",Bubba Watson,"Score: -6, Strokes: 202",Tony Finau,"Score: -1, Strokes: 199",Thomas Pieters,**MC** Score: 14 Strokes: 230,Tiger Woods,"Score: 4, Strokes: 220",Adam Scott,"Score: 2, Strokes: 218",1480.0,280.0
8,Chris Stine,Rory McIlroy,"Score: -9, Strokes: 187",Phil Mickelson,"Score: 7, Strokes: 223",Brian Harman,"Score: 7, Strokes: 223",Louis Oosthuizen,"Score: -3, Strokes: 202",Cameron Smith,"Score: -4, Strokes: 208",Tiger Woods,"Score: 4, Strokes: 220",Ryan Moore,"Score: 2, Strokes: 218",1481.0,280.0
9,Jeremy Szyba,Rickie Fowler,"Score: -8, Strokes: 197",Paul Casey,"Score: 2, Strokes: 218",Bubba Watson,"Score: -6, Strokes: 202",Louis Oosthuizen,"Score: -3, Strokes: 202",Jason Dufner,**MC** Score: 13 Strokes: 229,Jimmy Walker,"Score: -1, Strokes: 215",Zach Johnson,"Score: 2, Strokes: 218",1481.0,277.0
10,Joe Spallo,Rory McIlroy,"Score: -9, Strokes: 187",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -11, Strokes: 180",Kevin Chappell,**MC** Score: 16 Strokes: 232,Thomas Pieters,**MC** Score: 14 Strokes: 230,Adam Scott,"Score: 2, Strokes: 218",Bryson DeChambeau,"Score: 4, Strokes: 220",1482.0,277.0


In [60]:
df.to_csv("masters_results.csv")

# Tabula Fun

In [61]:
from tabula import read_pdf

In [62]:
df = read_pdf("/Users/jeremyszyba/Desktop/2018 Masters.pdf")
df = df.drop(df.index[10])
df

,Group 1,Group 2,Group 3,Group 4,Group 5,Field,Field.1,Field.2,Field.3
0,Dustin Johnson,Tommy Fleetwood,Bubba Watson,Branden Grace,Li Haotong,Ryan Armour,Mike Weir,Patton Kizzire,Lin Yuxin
1,Justin Thomas,Jason Day,Rafael Cabrera Bello,Louis Oosthuizen,Adam Hadwin,Daniel Berger,Bernd Wiesberger,Bernhard Langer,Doug Ghim
2,Jon Rahm,Paul Casey,Brian Harman,Patrick Cantlay,Kyle Stanley,Wesley Bryan,Danny Willett,Sandy Lyle,Russell Henley
3,Jordan Spieth,Alex Noren,Patrick Reed,Tony Finau,Cameron Smith,Angel Cabrera,Tiger Woods,Matt Parziale,Billy Horschel
4,Justin Rose,Henrik Stenson,Kevin Kisner,Ross Fisher,Thomas Pieters,Austin Cook,Ian Woosnam,Doc Redman,Larry Mize
5,Hideki Matsuyama,Marc Leishman,Xander Schauffele,Matthew Fitzpatrick,Satoshi Kodaira,Fred Couples,Yuta Ikdea,Ted Potter Jr.,Vijay Singh
6,Rory McIlroy,Tyrrell Hatton,Francesco Molinari,Daniel Berger,Dylan Frittelli,Bryson DeChambeau,Trevor Immelman,Charl Schwartzel,Jose Maria Olazabal
7,Rickie Fowler,Phil Mickelson,Charley Hoffman,Kevin Chappell,Chez Reavie,Harry Ellis,Zach Johnson,Adam Scott,Ryan Moore
8,Sergio Garcia,Pat Perez,Kiradech Aphibarnrat,Brendan Steele,Jason Dufner,Jhonattan Vegas,Martin Kaymer,Shubanhkar Sharma,Joaquin Niemann
9,Brooks Koepka,Matt Kuchar,Gary Woodland,Webb Simpson,Siwoo Kim,Jimmy Walker,Yusaku Miyazato,Webb Simpson,Mark O'Meara


In [63]:
print([item for item, count in collections.Counter(player_list).items() if count > 1] + ["brookskoepka", "ryanarmour"])

['danielberger', 'webbsimpson', 'brookskoepka', 'ryanarmour']
